In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
from pprint import pprint
#import zipfile
import time
#import json
import io
from IPython.display import FileLink
g = 0

eventdir = '../input/baseball-events-from-retrosheetorg/events/'
gamedir = '../input/baseball-events-from-retrosheetorg/games/'
rosterdir = '../input/baseball-events-from-retrosheetorg/Rosters/'
metadatadir = '../input/baseball-events-from-retrosheetorg/metadata/'

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#roster_columns_names = ['playerID', 'nameLast', 'nameFirst', 'batting_hand', 'throwing_hand', 'team', 'position']


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
print('Packages loaded!')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


Packages loaded!


# Read metadata

Teams, ballparks, biographical data

In [2]:
s = time.time()
print('reading metadata...')
print('\tteams...')
teams_df = pandas.read_csv(metadatadir+'TEAMABR.TXT', index_col='teamID')

print('\tnicknames...')
teams_nicknames_df = pandas.read_csv(metadatadir+'CurrentNames.csv')#, header=None)
teams_nicknames_df.index.name = 'nicknameID'

print('\tballparks...')
ballparks_df = pandas.read_csv(metadatadir+'parkcode.txt', index_col='PARKID')

ballparks_df.loc[:, 'START'] = pandas.to_datetime(ballparks_df['START'], errors='coerce')
ballparks_df.loc[:, 'END'] = pandas.to_datetime(ballparks_df['END'], errors='coerce')

print('\tbiodata...')
biodata_df = pandas.read_csv(metadatadir+'BIOFILE.txt')

biodata_df.columns = [x.replace("CEME ", "CEMETERY ").replace(" ", "_") for x in biodata_df.columns]
biodata_df.columns = [x.lower() for x in biodata_df.columns]


biodata_df = biodata_df.rename(columns = {'playerid': 'bioID', 'last': 'nameLast', 'first': 'nameFirstMiddle', 'play_debut': 'player_debut', 'play_lastgame': 'player_lastgame'})

# # biodata_df.sample(1).T
for thisdatecol in [x for x in biodata_df.columns if ('date' in x) or ('debut' in x) or ('lastgame' in x)]:
    biodata_df.loc[:, thisdatecol] = pandas.to_datetime(biodata_df[thisdatecol], errors='coerce')
biodata_df = biodata_df.assign(nameFirst = biodata_df['nameFirstMiddle'].apply(lambda x: str(x).split(' ')[0]))
biodata_df = biodata_df.assign(nameMiddle = biodata_df[biodata_df['nameFirstMiddle'].apply(lambda x: str(x).find(" ")) > -1]['nameFirstMiddle'].apply(lambda x: str(x).split(" ")[1]))
biodata_df.loc[:, 'nameMiddle'] = biodata_df['nameMiddle'].fillna('')

biodata_df.loc[biodata_df['hof'] == 'HOF', 'hof'] = True
biodata_df.loc[biodata_df['hof'] == 'NOT', 'hof'] = False
biodata_df.loc[:, 'hof'] = biodata_df['hof'].astype('bool')

biodata_df = biodata_df.assign(height_inches = biodata_df[(biodata_df['height'].notnull()) & (biodata_df['height'].apply(lambda x: len(str(x).split('-')) == 2))]['height'].apply(lambda x: float(str(x).split('-')[0])*12 + float(str(x).split('-')[1])))
biodata_df = biodata_df.assign(is_player = np.nan)
biodata_df.loc[biodata_df['player_debut'].notnull(), 'is_player'] = True
biodata_df.loc[biodata_df['player_debut'].isnull(), 'is_player'] = False


#print('fixing duplicate bioIDs...')
#biodata_df.groupby('bioID').size().sort_values(ascending=False)

biodata_df = biodata_df.set_index('bioID')
biodata_df = biodata_df.sort_index()

e = time.time()
g += e-s

print('Read {0:,.0f} biographical data entries in {1:.1f} seconds!'.format(len(biodata_df), e-s))

reading metadata...
	teams...
	nicknames...
	ballparks...
	biodata...
Read 21,913 biographical data entries in 0.7 seconds!


In [3]:
s = time.time()
print('saving output metadata...')
teams_df.to_csv('teams.csv', encoding='utf-8')
teams_nicknames_df.to_csv('teams_nicknames.csv', encoding='utf-8')
ballparks_df.to_csv('ballparks.csv', encoding='utf-8')
biodata_df.to_csv('biodata.csv', encoding='utf-8')

e = time.time()
g += e-s

print('Read and saved:')
print('\t{0:,.0f} teams'.format(len(teams_df)))
print('\t{0:,.0f} nicknames'.format(len(teams_nicknames_df)))
print('\t{0:,.0f} ballparks'.format(len(ballparks_df)))
print('\t{0:,.0f} biographies'.format(len(biodata_df)))
print('in {0:,.1f} seconds!'.format(e-s))

#teams_df
#teams_df[teams_df['yearLast'].apply(lambda x: int(x) == np.nan)]#.dtypes
#teams_df

saving output metadata...
Read and saved:
	152 teams
	123 nicknames
	260 ballparks
	21,913 biographies
in 0.6 seconds!


# Parse roster data

In [4]:
s = time.time()
print('Getting roster files...')
rosters_df = pandas.DataFrame()
roster_file_list = [rosterdir+x for x in os.listdir(rosterdir) if x[-4:] == '.ROS']


for i in range(0,len(roster_file_list)):
    if (np.mod(i, 500) == 0):
        print('\tReading roster file {0:,.0f} of {1:,.0f}...'.format(i, len(roster_file_list)))
    rosters_i_df = pandas.read_csv(roster_file_list[i], header=None, encoding='utf-8')
    rosters_i_df = rosters_i_df[:-1]  # remove stop character in last line of file
    rosters_i_df = rosters_i_df.assign(theyear = int(roster_file_list[i].split('/')[-1][3:7]))
    rosters_df = pandas.concat((rosters_df, rosters_i_df), axis=0)


roster_column_list = ['playerID', 'nameLast', 'nameFirst', 'bats', 'throws', 'theteam', 'position', 'theyear']
for j in range(0, len(rosters_df.columns[:-1])):
     rosters_df = rosters_df.rename(columns={j: roster_column_list[j]})

rosters_df = rosters_df.reset_index(drop=True)
# ## Find duplicates
#print(rosters_df.groupby(['playerID', 'theyear', 'theteam', 'position'], as_index=False).size().sort_values(by='size', ascending=False))
## Likely duplicates: chouf101/1914/BRF
print('Deleting duplicate entry: chouf101/1914/BRF/OF...')
rosters_df = rosters_df.drop(rosters_df[(rosters_df['playerID'] == 'chouf101') & (rosters_df['theyear'] == 1914) & (rosters_df['theteam'] == 'BRF') & (rosters_df['position'] == 'OF')].head(1).index, axis=0)
# ## Check that there are no more duplicates
#print(rosters_df.groupby(['playerID', 'theyear', 'theteam', 'position'], as_index=False).size().sort_values(by='size', ascending=False))

rosters_df = rosters_df.sort_values(by=['theyear', 'theteam', 'playerID', 'position'])
rosters_df = rosters_df.reset_index(drop=True)
rosters_df.index.name = 'rosterID'
rosters_df

print('Saving output rosters...')
rosters_df.to_csv('rosters-orig.csv', encoding='utf-8')

e = time.time()
g += e-s
print('\n')
print('Saved {0:,.0f} roster entries in {1:,.1f} seconds!'.format(len(rosters_df), e-s))
#rosters_df[rosters_df['playerID'].apply(lambda x: len(x) == 1)]

Getting roster files...
	Reading roster file 0 of 3,015...
	Reading roster file 500 of 3,015...
	Reading roster file 1,000 of 3,015...
	Reading roster file 1,500 of 3,015...
	Reading roster file 2,000 of 3,015...
	Reading roster file 2,500 of 3,015...
	Reading roster file 3,000 of 3,015...
Deleting duplicate entry: chouf101/1914/BRF/OF...
Saving output rosters...


Saved 109,100 roster entries in 42.9 seconds!


In [5]:
#FileLink(r'rosters-orig.csv')

# Get game data for use in events

In [6]:
s = time.time()

gamedata_df = pandas.DataFrame()

gamedirs = sorted([gamedir+x+'/' for x in os.listdir(gamedir)])
regularseasondir = [x for x in gamedirs if '']

filenames = []

for thisdir in gamedirs:
    filenames += sorted([thisdir+x for x in os.listdir(thisdir)])
# #filenames = sorted([gamedir+x for x in os.listdir(gamedir) if '1916' in x])

allstarfiles = ['../input/baseball-events-from-retrosheetorg/games/glwc/glwc.txt']
postfiles = [
    '../input/baseball-events-from-retrosheetorg/games/glwc/glwc.txt',
    '../input/baseball-events-from-retrosheetorg/games/gldv/gldv.txt', 
    '../input/baseball-events-from-retrosheetorg/games/gllc/gllc.txt',
    '../input/baseball-events-from-retrosheetorg/games/glws/glws.txt'
]
rsfiles = [x for x in filenames if x[-8:-6] in ['18', '19', '20']]
print('Found {0:,.0f} regular season gamefiles and {1:,.0f} allstar/postseason files...'.format(len(rsfiles), len(allstarfiles)+len(postfiles)))


print('Parsing games...')
for thisfile in rsfiles:
    thisyear = int(thisfile[-8:-4])
    if (np.mod(thisyear, 20) == 0):
        print('\tParsing regular season game logs from {0:.0f}...'.format(thisyear))
    gamedata_df = pandas.concat((gamedata_df, pandas.read_csv(thisfile, header=None, converters={19: lambda x: str(x), 20: lambda x: str(x)})), axis=0)
print('\tParsing all-star games...')
for thisfile in allstarfiles:
    gamedata_df = pandas.concat((gamedata_df, pandas.read_csv(thisfile, header=None, converters={19: lambda x: str(x), 20: lambda x: str(x)})), axis=0)
print('\tParsing postseason games...')
for thisfile in postfiles:
    gamedata_df = pandas.concat((gamedata_df, pandas.read_csv(thisfile, header=None, converters={19: lambda x: str(x), 20: lambda x: str(x)})), axis=0)



# Columns given at http://www.retrosheet.org/gamelogs/glfields.txt
colnames = ['date','gameno','dayofweek']
colnames += ['visitor','visitor_league','visitor_gameno']
colnames += ['home','home_league','home_gameno']
colnames += ['visitor_score','home_score']
colnames += ['nOuts','day_night','completed','forfeit','protest','parkID','attendance','length']
colnames += ['visitor_linescore','home_linescore']
colnames += ['visitor_ab','visitor_h','visitor_2b','visitor_3b','visitor_hr']
colnames += ['visitor_rbi','visitor_sac','visitor_sacfly','visitor_hbp']
colnames += ['visitor_bb','visitor_ibb','visitor_k','visitor_sb','visitor_cs']
colnames += ['visitor_gidp','visitor_ci','visitor_lob','visitor_nPitchers']
colnames += ['visitor_er_ind','visitor_er_team','visitor_wp','visitor_balk']
colnames += ['visitor_putout','visitor_assist','visitor_e','visitor_pb']
colnames += ['visitor_dp','visitor_tp','home_ab','home_h','home_2b','home_3b']
colnames += ['home_hr','home_rbi','home_sac','home_sacfly','home_hbp','home_bb']
colnames += ['home_ibb','home_k','home_sb','home_cs','home_gidp','home_ci']
colnames += ['home_lob','home_nPitchers','home_er_ind','home_er_team','home_wp']
colnames += ['home_balk','home_putout','home_assist','home_e','home_pb']
colnames += ['home_dp','home_tp']
colnames += ['ump_home_id','ump_home_name','ump_1b_id']
colnames += ['ump_1b_name','ump_2b_id','ump_2b_name','ump_3b_id','ump_3b_name']
colnames += ['ump_lf_id','ump_lf_name','ump_rf_id','ump_rf_name']
colnames += ['visitor_manager_id','visitor_manager_name','home_manager_id','home_manager_name']
colnames += ['winning_pitcher_id','winning_pitcher_name','losing_pitcher_id']
colnames += ['losing_pitcher_name','saving_pitcher_id','saving_pitcher_name']
colnames += ['gw_rbi_batter_id','gw_rbi_batter_name','visitor_starting_pitcher_id']
colnames += ['visitor_starting_pitcher_name','home_starting_pitcher_id']
colnames += ['home_starting_pitcher_name']
colnames += ['visitor_batter1_id','visitor_batter1_name','visitor_batter1_pos']
colnames += ['visitor_batter2_id','visitor_batter2_name','visitor_batter2_pos']
colnames += ['visitor_batter3_id','visitor_batter3_name','visitor_batter3_pos']
colnames += ['visitor_batter4_id','visitor_batter4_name','visitor_batter4_pos']
colnames += ['visitor_batter5_id','visitor_batter5_name','visitor_batter5_pos']
colnames += ['visitor_batter6_id','visitor_batter6_name','visitor_batter6_pos']
colnames += ['visitor_batter7_id','visitor_batter7_name','visitor_batter7_pos']
colnames += ['visitor_batter8_id','visitor_batter8_name','visitor_batter8_pos']
colnames += ['visitor_batter9_id','visitor_batter9_name','visitor_batter9_pos']
colnames += ['home_batter1_id','home_batter1_name','home_batter1_pos']
colnames += ['home_batter2_id','home_batter2_name','home_batter2_pos']
colnames += ['home_batter3_id','home_batter3_name','home_batter3_pos']
colnames += ['home_batter4_id','home_batter4_name','home_batter4_pos']
colnames += ['home_batter5_id','home_batter5_name','home_batter5_pos']
colnames += ['home_batter6_id','home_batter6_name','home_batter6_pos']
colnames += ['home_batter7_id','home_batter7_name','home_batter7_pos']
colnames += ['home_batter8_id','home_batter8_name','home_batter8_pos']
colnames += ['home_batter9_id','home_batter9_name','home_batter9_pos']
colnames += ['misc','acquisition']

gamedata_df.columns = colnames

print('Calculating gameID...')
gamedata_df = gamedata_df.assign(gameID = gamedata_df.apply(lambda row: row['home']+str(row['date'])+str(row['gameno']), axis=1))

print('Fixing data types...')
gamedata_df['date'] = pandas.to_datetime(gamedata_df['date'], format='%Y%m%d', errors='coerce')
gamedata_df['visitor_linescore'] = '"'+gamedata_df['visitor_linescore'].astype('str')+'"'
gamedata_df['home_linescore'] = '"'+gamedata_df['home_linescore'].astype('str')+'"'
gamedata_df.loc[gamedata_df['visitor_linescore'] == 'nan', 'visitor_linescore'] = 'n/a'
gamedata_df.loc[gamedata_df['home_linescore'] == 'nan', 'home_linescore'] = 'n/a'

gamedata_df = gamedata_df.set_index('gameID')

print('Saving games...')
gamedata_df.to_csv('games.csv', encoding='utf-8')

e = time.time()
print('\nParsed and saved {0:,.0f} game logs in {1:,.0f} seconds!'.format(len(gamedata_df), e-s))
    
gamedata_df.sample(3).T
# #gamedata_df.loc['BOS191604120'][['visitor_linescore', 'home_linescore']]#.dtypes
# # #colnames[19:21]


Found 152 regular season gamefiles and 5 allstar/postseason files...
Parsing games...
	Parsing regular season game logs from 1880...
	Parsing regular season game logs from 1900...
	Parsing regular season game logs from 1920...
	Parsing regular season game logs from 1940...
	Parsing regular season game logs from 1960...
	Parsing regular season game logs from 1980...
	Parsing regular season game logs from 2000...
	Parsing regular season game logs from 2020...
	Parsing all-star games...
	Parsing postseason games...
Calculating gameID...
Fixing data types...
Saving games...

Parsed and saved 228,146 game logs in 55 seconds!


gameID,NYA195509290,MON197308220,OAK197806050
date,1955-09-29 00:00:00,1973-08-22 00:00:00,1978-06-05 00:00:00
gameno,0,0,0
dayofweek,Thu,Wed,Mon
visitor,BRO,SFN,BOS
visitor_league,NL,NL,AL
...,...,...,...
home_batter9_id,byrnt101,roges001,edwam101
home_batter9_name,Tommy Byrne,Steve Rogers,Mike Edwards
home_batter9_pos,1.0,1.0,4.0
misc,NaN,NaN,NaN


In [7]:
#FileLink(r'games.csv')